In [1]:
import backtrader as bt
import datetime
import pandas as pd
import math

In [2]:
# import pymysql


# # Connect to the database
# connection = pymysql.connect(host='localhost',
#                              port=3306,
#                              user='root',
#                              password='seiRaefoe9jeufooT1uipei5gungiFah',
#                              db='stock_etf',
#                              charset='utf8mb4',
#                              cursorclass=pymysql.cursors.SSDictCursor)


# try:
#     with connection.cursor() as cursor:
#         # Read a single record
#         sql = "SELECT * FROM `etf159920_15m` WHERE datetime BETWEEN %s AND %s"
#         cursor.execute(sql, (datetime.date(2019,1,1),datetime.date(2020,5,1)) )
#         result = cursor.fetchall()
# finally:
#     connection.close()

# df = pd.DataFrame(data=result)
# df = df.set_index(['datetime'])
# df

In [3]:
from lutils.stock import LTdxHq

In [4]:
ltdxhq = LTdxHq()

In [5]:
# ll = ltdxhq.stock_list()

In [6]:
# ll[(ll['pre_close'] > 30) & (ll['pre_close'] < 50)]

In [7]:
# 399300
code = '603636' # 000032 300142 603636 600519 688567
# df = ltdxhq.get_k_data_daily('300142') # 000032 300142 603636 600519
# df = ltdxhq.get_k_data_15min(code) #' # 000032 300142 603636 600519
# df = ltdxhq.to_qfq(code, df)

In [8]:
# df.index = df.index.unique(level=1)

In [9]:
df15 = ltdxhq.get_k_data_15min(code) # 000032 300142 603636 600519
df15 = ltdxhq.to_qfq(code, df15)

df60 = ltdxhq.get_k_data_1hour(code) # 000032 300142 603636 600519
df60 = ltdxhq.to_qfq(code, df60)

df1d = ltdxhq.get_k_data_daily(code) # 000032 300142 603636 600519
df1d = ltdxhq.to_qfq(code, df1d)

In [10]:
df15.index = df15.index.unique(level=1)
df60.index = df60.index.unique(level=1)
df1d.index = df1d.index.unique(level=0)

In [11]:
ltdxhq.close()

In [12]:
df15.index = pd.to_datetime(df15.index)
df60.index = pd.to_datetime(df60.index)
df1d.index = pd.to_datetime(df1d.index)

In [13]:
data15 = df15['2021-01-01':]
data60 = df60['2021-01-01':]
data1d = df1d['2021-01-01':]

In [14]:
data1d

,open,close,high,low,volume,amount
date,,,,,,
2021-01-04,9.179221,9.287792,9.297662,9.090390,4484555.0,41926596.0
2021-01-05,9.307532,9.317403,9.435844,9.277922,3896723.0,36895040.0
2021-01-06,9.258182,9.159481,9.297662,9.110130,3874385.0,36067080.0
2021-01-07,9.149610,8.843636,9.149610,8.685714,5851525.0,52623452.0
2021-01-08,8.804156,8.912727,9.120000,8.646234,4604155.0,41699760.0
...,...,...,...,...,...,...
2021-09-27,8.920000,8.640000,8.970000,8.620000,5689958.0,49751368.0
2021-09-28,8.640000,8.550000,8.650000,8.490000,3466596.0,29651572.0
2021-09-29,8.520000,8.280000,8.520000,8.230000,4925307.0,41109280.0


In [15]:
import pandas as pd
import array
import talib as ta
'''
自定义的CCI计算指标类，继承backtrader的Indicator(注意：不是indicator)
'''

class CCI(bt.Indicator):
    # 定义CCI lines，类中可以使用self.lines.CCI进行访问，也可以使用self.l.CCI进行访问
    # 注意以下self.lines.CCI是linebuffer类型，在init中使用self.lines.CCI，而在next中就需要使用
    # self.lines.CCI[index]的形式
    lines = ('CCI',)

    # kdj周期参数定义，可以使用self.params.timeperiod的形式访问，同样也可以按照self.p.timeperiod的形式访问
    # params = (('timeperiod', 14))

    def __init__(self, **kwargs):
        super(CCI, self).__init__()
        # 将策略传入的周期参数进行赋值
        # self.params.fastperiod=fastperiod
        # self.params.slowperiod=slowperiod
        # self.params.signalperiod=signalperiod
        # 组装变量
        timeperiod = None
        keys = kwargs.keys()
        for k in keys:
            if k == 'timeperiod':
                timeperiod = kwargs[k]

        # 将数据转成Series类型，且包含最低价、最高价及收盘价
        # 计算CCI
        cci = ta.CCI(pd.Series(self.data.high.array),
                     pd.Series(self.data.low.array),
                     pd.Series(self.data.close.array),
                     timeperiod=timeperiod or 14)
        # 将序列赋值给lines的array,在Strategy中可以通过访问lines进行访问下面数据
        self.lines.CCI.array = array.array(str('d'), list(cci.values))

In [16]:
class MultiTFStrategy(bt.Strategy):
    params = (
        ('macd1', 12),
        ('macd2', 26),
        ('macdsig', 9),
        ('atrperiod', 14),  # ATR Period (standard)
        ('atrdist', 3.0),   # ATR distance for stop price
        ('smaperiod', 30),  # SMA Period (pretty standard)
        ('dirperiod', 10),  # Lookback period to consider SMA trend direction
        
        ('short', 30),
        ('long', 70),
    )
    
    # states defination
    Empty, M15Hold, H1Hold, D1Hold = range(4)
    States = [
        'Empty', 'M15Hold', 'H1Hold', 'D1Hold',
    ]
    
    def log(self, txt):
        ''' Logging function for this strategy'''
        dt = self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):
        self.ma15m = bt.indicators.MACD(self.dnames.hs15m,
                                       period_me1=self.p.macd1,
                                       period_me2=self.p.macd2,
                                       period_signal=self.p.macdsig, plot=False)
        
        
        self.ma1h = bt.indicators.MACD(self.dnames.hs1h,
                                       period_me1=self.p.macd1,
                                       period_me2=self.p.macd2,
                                       period_signal=self.p.macdsig, plot=False)
        
        self.ma1d = bt.indicators.MACD(self.dnames.hs1d,
                                       period_me1=self.p.macd1,
                                       period_me2=self.p.macd2,
                                       period_signal=self.p.macdsig, plot=False)
        
#         self.rsi15 = bt.indicators.RSI_EMA(self.dnames.hs15m.close, period=14) # RSI_SMA
#         self.rsi60 = bt.indicators.RSI_EMA(self.dnames.hs1h.close, period=14)
#         self.rsi1d = bt.indicators.RSI_EMA(self.dnames.hs1d.close, period=14)
    
#         self.rsi15 = bt.indicators.RSI_EMA(self.datas[0].close, period=21)
#         self.rsi60 = bt.indicators.RSI_EMA(self.datas[1].close, period=21)
#         self.rsi1d = bt.indicators.RSI_EMA(self.datas[2].close, period=21)
        
#         self.ADX15 = bt.indicators.DirectionalMovementIndex(self.dnames.hs15m, period=12)
#         self.CCIplusDI15 = bt.indicators.CommodityChannelIndex(self.ADX15.DIplus, period=20)
#         self.CCIminusDI15 = bt.indicators.CommodityChannelIndex(self.ADX15.DIminus, period=20)
        
#         self.ADX60 = bt.indicators.DirectionalMovementIndex(self.dnames.hs1h, period=12)
#         self.CCIplusDI60 = bt.indicators.CommodityChannelIndex(self.ADX60.DIplus, period=20)
#         self.CCIminusDI60 = bt.indicators.CommodityChannelIndex(self.ADX60.DIminus, period=20)
        
#         self.ADX1d = bt.indicators.DirectionalMovementIndex(self.dnames.hs1d, period=12)
#         self.CCIplusDI1d = bt.indicators.CommodityChannelIndex(self.ADX1d.DIplus, period=20)
#         self.CCIminusDI1d = bt.indicators.CommodityChannelIndex(self.ADX1d.DIminus, period=20)
        
    
        self.c15m = bt.indicators.CrossOver(self.ma15m.macd, self.ma15m.signal, plot=False)
        self.c1h = bt.indicators.CrossOver(self.ma1h.macd, self.ma1h.signal, plot=False)
        self.c1d = bt.indicators.CrossOver(self.ma1d.macd, self.ma1d.signal, plot=False)
        
        self.bsig15m = self.c15m == 1
        self.bsig1h = self.c1h == 1
        self.bsig1d = self.c1d == 1
        self.sell_signal = self.c1d == -1
        
        
        self.st = self.Empty
        self.st_map = {
            self.Empty: self._empty,
            self.M15Hold: self._m15hold,
            self.H1Hold: self._h1hold,
            self.D1Hold: self._d1hold,
        }
        
        # To keep track of pending orders
        self.order = None
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status == order.Completed:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, St: %s, Size: %d, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (
                        self.States[self.st],
                        order.executed.size,
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                    )
                )

            else:  # Sell
                self.log(
                    'SELL EXECUTED, St: %s, Size: %d, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (
                        self.States[self.st],
                        order.executed.size,
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm
                    )
                )

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        # just call state_map function
        self.order = self.st_map[self.st]()
        
        # Check if we are in the market and no buy order issued
        if self.position and not self.order:
            # Already in the market ... we might sell
            if self.sell_signal: # and self.rsi1d > self.params.long:
                self.st = self.Empty
                # Keep track of the created order to avoid a 2nd order
                self.order = self.close()
                
    def _empty(self):
#         print(self.rsi15)
        if self.bsig15m: # and self.rsi15 < self.params.short: # and self.CCIplusDI15[-1] > 100 and self.CCIminusDI15[-1] < 100 # self.c15m > .7 and self.rsi15 < self.params.short: #
            price = self.data0.close[0]
            cash = self.broker.get_cash()
            # 20% of the cash
            share = int(math.floor((0.2*cash)/price))

            # set state
            self.st = self.M15Hold
            return self.buy(size=share)
        
    def _m15hold(self):
        if self.bsig1h: # and self.rsi60 < self.params.short: # and self.CCIplusDI60[-1] > 100 and self.CCIminusDI60[-1] < 100 # self.c1h > .7 and self.rsi60 < self.params.short: #
            price = self.data0.close[0]
            cash = self.broker.get_cash()
            # half of the remain cash ( 60% )
            share = int(math.floor((0.6*cash)/price))
            
            # set state
            self.st = self.H1Hold
            return self.buy(size=share)
        
    def _h1hold(self):
        if self.bsig1d: # and self.rsi1d < self.params.short: # and self.CCIplusDI1d[-1] > 100 and self.CCIminusDI1d[-1] < 100 # self.c1d > .7 and self.rsi1d < self.params.short: #
            price = self.data0.close[0]
            cash = self.broker.get_cash()
            # half of the remain cash (80%)
            share = int(math.floor((0.8*cash)/price))
            
            # set state
            self.st = self.D1Hold
            return self.buy(size=share)
        
    def _d1hold(self):
        return None

In [17]:
cerebro = bt.Cerebro(oldtrades=True)

feed15 = bt.feeds.PandasData(dataname=data15, openinterest=None, compression=15, timeframe=bt.TimeFrame.Minutes)
feed60 = bt.feeds.PandasData(dataname=data60, openinterest=None, compression=60, timeframe=bt.TimeFrame.Minutes)
feed1d = bt.feeds.PandasData(dataname=data1d, openinterest=None, compression=1, timeframe=bt.TimeFrame.Days)

cerebro.adddata(feed15, name='hs15m')
cerebro.adddata(feed60, name='hs1h')
cerebro.adddata(feed1d, name='hs1d')

# cerebro.resampledata(feed15, name='hs1h', timeframe=bt.TimeFrame.Minutes, compression=60)
# cerebro.resampledata(feed15, name='hs1d', timeframe=bt.TimeFrame.Days)

cerebro.addstrategy(MultiTFStrategy)

# 小场面1万起始资金
cerebro.broker.setcash(10000.0)

# 手续费万5
cerebro.broker.setcommission(0.0005)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

result = cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
False
False
False
False
False
False
False
False
False
False
False
False
False
True
2021-02-26T14:30:00, BUY EXECUTED, St: M15Hold, Size: 246, Price: 8.11, Cost: 1996.07, Comm 1.00
2021-03-01T13:15:00, BUY EXECUTED, St: H1Hold, Size: 581, Price: 8.25, Cost: 4794.27, Comm 2.40
2021-03-09T09:45:00, SELL EXECUTED, St: Empty, Size: -827, Price: 8.19, Cost: 6790.34, Comm 3.39
2021-03-09T09:45:00, OPERATION PROFIT, GROSS -19.71, NET -26.49
False
False
False
False
False
False
False
True
2021-03-09T13:15:00, BUY EXECUTED, St: M15Hold, Size: 245, Price: 8.15, Cost: 1997.87, Comm 1.00
2021-03-09T13:30:00, SELL EXECUTED, St: Empty, Size: -245, Price: 8.18, Cost: 1997.87, Comm 1.00
2021-03-09T13:30:00, OPERATION PROFIT, GROSS 5.95, NET 3.95
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
2021-03-10T13:15:00, BUY EXECUTED, St: M15Hold, Size: 250, Price: 7.98, Cost: 1996.13, Comm 1.00
2021-03-16T13:15:00, BUY EXECUTED, 

In [18]:
cerebro.broker

In [18]:
cerebro.plot(
    iplot=False,
    start=datetime.date(2021, 1, 1),
    end=datetime.date(2021, 9, 30),
    style='candlestick',
    barup='red',
    bardown='green',
)

[[<Figure size 1707x960 with 8 Axes>]]